In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms


model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Reemplazar última capa para 4 clases
model.fc = nn.Linear(model.fc.in_features, 4)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(device)

cuda


In [3]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [5]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

import sys
import os
sys.path.append(os.path.abspath(".."))
from src.dataset import EcommerceDataset

dataset = EcommerceDataset(
    csv_path="../data/fashion.csv",
    image_root_dir="../data",
    transform=transform
)

train_dataset = EcommerceDataset(
    csv_path="../data/fashion.csv",
    image_root_dir="../data",
    transform=train_transform
)

val_dataset = EcommerceDataset(
    csv_path="../data/fashion.csv",
    image_root_dir="../data",
    transform=val_transform
)

from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

labels = (dataset.df["Category"] + "_" + dataset.df["Gender"]).values
class_to_idx = {
    "Apparel_Boys": 0,
    "Apparel_Girls": 1,
    "Footwear_Men": 2,
    "Footwear_Women": 3
}

labels = [class_to_idx[x] for x in labels]
indices = list(range(len(dataset)))

train_idx, val_idx = train_test_split(
    indices,
    test_size=0.2,
    stratify=labels,
    random_state=42
)

train_dataset = Subset(train_dataset, train_idx)
val_dataset = Subset(val_dataset, val_idx)

from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0  # Windows safe
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0
)
unique_labels = set()

unique_labels = set()
for _, labels in train_loader:
    unique_labels.update(labels.tolist())

print(unique_labels)

{0, 1, 2, 3}


In [12]:
for param in model.parameters():
    param.requires_grad = False

# 1️⃣ Crear modelo
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Primero congelamos todo
for param in model.parameters():
    param.requires_grad = False

# Reemplazamos la última capa
model.fc = nn.Linear(model.fc.in_features, 4)

# Descongelamos layer4
for param in model.layer4.parameters():
    param.requires_grad = True

# También la fc obviamente
for param in model.fc.parameters():
    param.requires_grad = True


# 4️⃣ Mover a device
model = model.to(device)

# 5️⃣ Crear optimizer DESPUÉS
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-5
)

num_epochs = 15

best_val_acc = 0.0
best_val_loss = float("inf")
patience = 3
counter = 0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    # ----- VALIDATION -----
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_acc = correct / total

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), "best_model.pth")
        print("✅ Modelo guardado (mejor hasta ahora)")
    else:
        counter += 1
        print(f"⚠️ No mejora en {counter} epoch(s)")
        
        if counter >= patience:
            print("🛑 Early stopping activado")
            break

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {avg_train_loss:.4f}")
    print(f"Val Loss: {avg_val_loss:.4f}")
    print(f"Val Acc: {val_acc:.4f}")
    print("-"*30)

✅ Modelo guardado (mejor hasta ahora)
Epoch [1/15]
Train Loss: 1.0308
Val Loss: 0.8199
Val Acc: 0.6753
------------------------------
✅ Modelo guardado (mejor hasta ahora)
Epoch [2/15]
Train Loss: 0.6797
Val Loss: 0.6506
Val Acc: 0.7491
------------------------------
✅ Modelo guardado (mejor hasta ahora)
Epoch [3/15]
Train Loss: 0.5519
Val Loss: 0.5598
Val Acc: 0.7801
------------------------------
✅ Modelo guardado (mejor hasta ahora)
Epoch [4/15]
Train Loss: 0.4660
Val Loss: 0.5235
Val Acc: 0.7921
------------------------------
✅ Modelo guardado (mejor hasta ahora)
Epoch [5/15]
Train Loss: 0.4185
Val Loss: 0.4594
Val Acc: 0.8162
------------------------------
✅ Modelo guardado (mejor hasta ahora)
Epoch [6/15]
Train Loss: 0.3836
Val Loss: 0.4288
Val Acc: 0.8282
------------------------------
✅ Modelo guardado (mejor hasta ahora)
Epoch [7/15]
Train Loss: 0.3533
Val Loss: 0.3967
Val Acc: 0.8591
------------------------------
✅ Modelo guardado (mejor hasta ahora)
Epoch [8/15]
Train Loss:

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds))
print(confusion_matrix(all_labels, all_preds))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92       152
           1       0.88      0.90      0.89       114
           2       0.86      0.95      0.90       162
           3       0.94      0.83      0.88       154

    accuracy                           0.90       582
   macro avg       0.90      0.90      0.90       582
weighted avg       0.90      0.90      0.90       582

[[139  13   0   0]
 [ 11 103   0   0]
 [  0   0 154   8]
 [  0   1  25 128]]


: 

In [8]:
(dataset.df["Category"] + "_" + dataset.df["Gender"]).value_counts()

Footwear_Men      811
Footwear_Women    769
Apparel_Boys      759
Apparel_Girls     567
Name: count, dtype: int64

In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix


cm = confusion_matrix(all_labels, all_preds)

class_names = ["Apparel_Boys", "Apparel_Girls", 
               "Footwear_Men", "Footwear_Women"]

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.savefig("confusion_matrix3.png")
plt.close()

Chequeo que no hay interseccion entre los datos de tests y entrenamiento

In [6]:
train_set = set(train_idx)
val_set = set(val_idx)

print("Intersección:", train_set.intersection(val_set))

Intersección: set()


In [9]:
train_paths = set(dataset.df.iloc[train_idx]["Image"])
val_paths   = set(dataset.df.iloc[val_idx]["Image"])

print("Intersección real:", train_paths.intersection(val_paths))

Intersección real: set()


In [10]:
train_products = set(dataset.df.iloc[train_idx]["ProductId"])
val_products   = set(dataset.df.iloc[val_idx]["ProductId"])

print(train_products.intersection(val_products))

set()
